In [1]:
from vit_prisma.sae import VisionModelSAERunnerConfig
from vit_prisma.sae import VisionSAETrainer
from vit_prisma.transforms import get_clip_val_transforms


import torchvision
import torch

from torch.utils.data import DataLoader, Subset


/data/xgao/anaconda3/envs/sae/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import os
# os.environ['HTTP_PROXY'] = 'http://127.0.0.1:17890'
# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:17890'
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,2,3"

# Train the SAE

In [3]:

# Load an SAE
from huggingface_hub import hf_hub_download, list_repo_files
from vit_prisma.sae import SparseAutoencoder

def load_sae(repo_id, file_name, config_name):
    # Step 1: Download SAE weights and SAE config from Hugginface
    sae_path = hf_hub_download(repo_id, file_name) # Download SAE weights
    hf_hub_download(repo_id, config_name) # Download SAE config

    # Step 2: Now load the pretrained SAE weights from where you just downloaded them
    print(f"Loading SAE from {sae_path}...")
    sae = SparseAutoencoder.load_from_pretrained(sae_path) # This now automatically gets the config.json file in that folder and converts it into a VisionSAERunnerConfig object
    return sae

# Change the repo_id to the Huggingface repo of your chosen SAE. See /docs for a list of SAEs.
repo_id = "Prisma-Multimodal/sparse-autoencoder-clip-b-32-sae-vanilla-x64-layer-10-hook_mlp_out-l1-1e-05" 

file_name = "weights.pt" # Usually weights.pt but may have slight naming variation. See the chosen HF repo for the exact file name
config_name = "config.json"

sae = load_sae(repo_id, file_name, config_name)

2025-06-22 05:21:53 DEBUG:urllib3.connectionpool: Starting new HTTPS connection (1): huggingface.co:443
2025-06-22 05:21:54 DEBUG:urllib3.connectionpool: https://huggingface.co:443 "HEAD /Prisma-Multimodal/sparse-autoencoder-clip-b-32-sae-vanilla-x64-layer-10-hook_mlp_out-l1-1e-05/resolve/main/weights.pt HTTP/1.1" 302 0
2025-06-22 05:21:54 DEBUG:urllib3.connectionpool: https://huggingface.co:443 "HEAD /Prisma-Multimodal/sparse-autoencoder-clip-b-32-sae-vanilla-x64-layer-10-hook_mlp_out-l1-1e-05/resolve/main/config.json HTTP/1.1" 200 0


Loading SAE from /data/xgao/.cache/huggingface/hub/models--Prisma-Multimodal--sparse-autoencoder-clip-b-32-sae-vanilla-x64-layer-10-hook_mlp_out-l1-1e-05/snapshots/b46d6e9d6c114a53364c5c172ea5023e0e52d4e2/weights.pt...


2025-06-22 05:21:55 WARNING:root: Deprecated field 'total_training_images' found in config. It will be ignored.
2025-06-22 05:21:55 WARNING:root: Deprecated field 'total_training_tokens' found in config. It will be ignored.
2025-06-22 05:21:55 WARNING:root: Deprecated field 'd_sae' found in config. It will be ignored.
2025-06-22 05:21:55 INFO:root: n_tokens_per_buffer (millions): 0.032
2025-06-22 05:21:55 INFO:root: Lower bound: n_contexts_per_buffer (millions): 0.00064
2025-06-22 05:21:55 INFO:root: Total training steps: 158691
2025-06-22 05:21:55 INFO:root: Total training images: 13000000
2025-06-22 05:21:55 INFO:root: Total wandb updates: 1586
2025-06-22 05:21:55 INFO:root: Expansion factor: 64
2025-06-22 05:21:55 INFO:root: n_tokens_per_feature_sampling_window (millions): 204.8
2025-06-22 05:21:55 INFO:root: n_tokens_per_dead_feature_window (millions): 1024.0
2025-06-22 05:21:55 INFO:root: We will reset the sparsity calculation 158 times.
2025-06-22 05:21:55 INFO:root: Number token

In [4]:
from vit_prisma.models.model_loader import load_hooked_model

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = sae.cfg.model_name
model = load_hooked_model(model_name)
model.to(DEVICE) # Move to device

2025-06-22 05:21:55 INFO:root: Model 'open-clip:laion/CLIP-ViT-B-32-DataComp.XL-s13B-b90K' is supported and passes tests.
2025-06-22 05:21:55 INFO:root: model_id download_pretrained_from_hf: laion/CLIP-ViT-B-32-DataComp.XL-s13B-b90K
2025-06-22 05:21:55 DEBUG:urllib3.connectionpool: https://huggingface.co:443 "HEAD /laion/CLIP-ViT-B-32-DataComp.XL-s13B-b90K/resolve/main/open_clip_config.json HTTP/1.1" 200 0
2025-06-22 05:21:56 INFO:root: model_id download_pretrained_from_hf: laion/CLIP-ViT-B-32-DataComp.XL-s13B-b90K
2025-06-22 05:21:56 DEBUG:urllib3.connectionpool: https://huggingface.co:443 "HEAD /laion/CLIP-ViT-B-32-DataComp.XL-s13B-b90K/resolve/main/open_clip_pytorch_model.bin HTTP/1.1" 302 0
2025-06-22 05:21:56 INFO:root: visual projection shape: torch.Size([768, 512])
2025-06-22 05:21:57 INFO:root: Loaded pretrained model open-clip:laion/CLIP-ViT-B-32-DataComp.XL-s13B-b90K into HookedTransformer


HookedViT(
  (embed): PatchEmbedding(
    (proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
  )
  (hook_embed): HookPoint()
  (pos_embed): PosEmbedding()
  (hook_pos_embed): HookPoint()
  (hook_full_embed): HookPoint()
  (ln_pre): LayerNorm(
    (hook_scale): HookPoint()
    (hook_normalized): HookPoint()
  )
  (hook_ln_pre): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): Ho

In [5]:
# Put your ImageNet Paths here
from vit_prisma.transforms import get_clip_val_transforms
from torchvision.datasets import CocoDetection

# imagenet_train_path = '/network/scratch/s/sonia.joseph/datasets/kaggle_datasets/ILSVRC/Data/CLS-LOC/train'
# imagenet_validation_path = '/network/scratch/s/sonia.joseph/datasets/kaggle_datasets/ILSVRC/Data/CLS-LOC/val'

# data_transforms = get_clip_val_transforms()
# train_dataset = torchvision.datasets.ImageFolder(imagenet_train_path, transform=data_transforms)
# eval_dataset = torchvision.datasets.ImageFolder(imagenet_validation_path, transform=data_transforms)

mscoco_root = '/data/wyx/datasets/COCO'
mscoco_train_img_dir = f'{mscoco_root}/train2017'
mscoco_val_img_dir = f'{mscoco_root}/val2017'
mscoco_train_ann_file = f'{mscoco_root}/annotations/instances_train2017.json'
mscoco_val_ann_file = f'{mscoco_root}/annotations/instances_val2017.json'

data_transforms = get_clip_val_transforms()
train_dataset = CocoDetection(mscoco_train_img_dir, mscoco_train_ann_file, transform=data_transforms)
eval_dataset = CocoDetection(mscoco_val_img_dir, mscoco_val_ann_file, transform=data_transforms)

loading annotations into memory...
Done (t=16.63s)
creating index...
index created!
loading annotations into memory...
Done (t=0.55s)
creating index...
index created!


In [6]:
sae_trainer_cfg = VisionModelSAERunnerConfig( 
    hook_point_layer=10,
    layer_subtype='hook_resid_post',
    dataset_name="mscoco",
    feature_sampling_window=1000,
    activation_fn_str='relu',
    checkpoint_path = '/data/xgao/code/interpretability/VIT-PRISMA/ckpts',
    log_to_wandb = True
)

2025-06-22 05:22:16 INFO:root: n_tokens_per_buffer (millions): 0.032
2025-06-22 05:22:16 INFO:root: Lower bound: n_contexts_per_buffer (millions): 0.00064
2025-06-22 05:22:16 INFO:root: Total training steps: 15869
2025-06-22 05:22:16 INFO:root: Total training images: 1300000
2025-06-22 05:22:16 INFO:root: Total wandb updates: 1586
2025-06-22 05:22:16 INFO:root: Expansion factor: 16
2025-06-22 05:22:16 INFO:root: n_tokens_per_feature_sampling_window (millions): 204.8
2025-06-22 05:22:16 INFO:root: n_tokens_per_dead_feature_window (millions): 1024.0
2025-06-22 05:22:16 INFO:root: We will reset the sparsity calculation 15 times.
2025-06-22 05:22:16 INFO:root: Number tokens in sparsity calculation window: 4.10e+06
2025-06-22 05:22:16 INFO:root: Gradient clipping with max_norm=1.0
2025-06-22 05:22:16 INFO:root: Using SAE initialization method: independent


In [7]:
import wandb
wandb.init(project="test", mode="online")  # 或 mode="online" 测试


2025-06-22 05:22:16 DEBUG:git.cmd: Popen(['git', 'rev-parse', '--show-toplevel'], cwd=/data/xgao/code/interpretability/ViT-Prisma, stdin=None, shell=False, universal_newlines=False)
2025-06-22 05:22:16 DEBUG:urllib3.connectionpool: Starting new HTTPS connection (1): api.wandb.ai:443
2025-06-22 05:22:16 DEBUG:urllib3.connectionpool: https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
2025-06-22 05:22:16 DEBUG:urllib3.connectionpool: https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
wandb: Currently logged in as: gaox23 (gaox23-fdu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
2025-06-22 05:22:17 DEBUG:git.cmd: Popen(['git', 'cat-file', '--batch-check'], cwd=/data/xgao/code/interpretability/ViT-Prisma, stdin=<valid stream>, shell=False, universal_newlines=False)


In [8]:
!unset HTTP_PROXY
!unset HTTPS_PROXY
import os
os.environ.pop("HTTP_PROXY", None)
os.environ.pop("HTTPS_PROXY", None)
!printenv HTTP_PROXY
!printenv HTTPS_PROXY

In [ ]:
trainer = VisionSAETrainer(sae_trainer_cfg, model, train_dataset, eval_dataset)
sae = trainer.run()


2025-06-22 05:22:19 INFO:root: get_activation_fn received: activation_fn=relu, kwargs={}


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


2025-06-22 05:22:23 DEBUG:asyncio: Using selector: EpollSelector


2025-06-22 05:22:23 DEBUG:git.cmd: Popen(['git', 'cat-file', '--batch-check'], cwd=/data/xgao/code/interpretability/ViT-Prisma, stdin=<valid stream>, shell=False, universal_newlines=False)


Objective value: 478317.9688:   1%|          | 2/200 [00:00<00:02, 85.45it/s]
Training SAE: Loss: 0.0175, MSE Loss: 0.0136, L1 Loss: 0.0039, L0: 11.0593:   8%|▊         | 5181440/65000000 [03:24<39:41, 25119.77it/s]2025-06-22 05:25:54 DEBUG:PIL.PngImagePlugin: STREAM b'IHDR' 16 13
2025-06-22 05:25:54 DEBUG:PIL.PngImagePlugin: STREAM b'IDAT' 41 32768
Training SAE: Loss: 0.0174, MSE Loss: 0.0135, L1 Loss: 0.0039, L0: 10.8109:   9%|▉         | 6172672/65000000 [04:04<39:37, 24738.74it/s]

...

Training SAE: Loss: 0.0156, MSE Loss: 0.0113, L1 Loss: 0.0043, L0: 11.7209:  90%|████████▉ | 58421248/65000000 [38:24<04:16, 25627.85it/s]

Saved SAE to /data/xgao/code/interpretability/VIT-PRISMA/ckpts/267a5b83-tinyclip_sae_16_hyperparam_sweep_lr/n_images_1170063.pt


2025-06-22 06:01:05 DEBUG:PIL.PngImagePlugin: STREAM b'IHDR' 16 13
2025-06-22 06:01:05 DEBUG:PIL.PngImagePlugin: STREAM b'IDAT' 41 32768
Training SAE: Loss: 0.0156, MSE Loss: 0.0113, L1 Loss: 0.0043, L0: 11.6630:  99%|█████████▉| 64462848/65000000 [42:21<00:20, 25697.27it/s]2025-06-22 06:04:52 DEBUG:PIL.PngImagePlugin: STREAM b'IHDR' 16 13
2025-06-22 06:04:52 DEBUG:PIL.PngImagePlugin: STREAM b'IDAT' 41 32768
Training SAE: Loss: 0.0156, MSE Loss: 0.0113, L1 Loss: 0.0043, L0: 11.7430: : 65003520it [42:42, 25365.47it/s]                            

Saved SAE to /data/xgao/code/interpretability/VIT-PRISMA/ckpts/267a5b83-tinyclip_sae_16_hyperparam_sweep_lr/n_images_1300070.pt


: 